In [1]:
pip install asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa/PR')
rows = cw.query_produrements()

urls = []
for row in rows:
    url = 'https://{base}/portaltransparencia-api/api/licitacoes/arquivos?entidade=1&exercicio={year}&tipoLicitacao={type}&licitacao={num}'.format(
        base=cw._url,
        year=row[2],
        type=row[3],
        num=row[4]
    )
    urls.append(url)

Scraping.crawlers.crawler_oxy


ModuleNotFoundError: No module named 'ironpdf'

In [ ]:
import asyncio
import aiohttp
import nest_asyncio

nest_asyncio.apply()

responses = []

async def fetch(client, url):
    async with client.get(url) as resp:
        assert resp.status == 200
        html = await resp.json()
        responses.append(html)


async def main():
    async with aiohttp.ClientSession() as client:
        await asyncio.gather(*[
            asyncio.ensure_future(fetch(client, url))
            for url in urls
        ])


loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [ ]:
pip install nest-asyncio


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
i=0
to_insert = []
for response in responses:
    i += 1
    for item in response:
        id = str(item['licitacao']) + '/' + str(item['exercicio']) + '/' + str(item['tipoLicitacao'])
        to_insert.append((cw._id_ibge, id, item['arquivoCompras']['idArquivo'], os.path.join("/" + str(cw._id_ibge), id, str(item['arquivoCompras']['idArquivo']) + ".pdf"), item['arquivoCompras']['nomeArquivo'], item['arquivoCompras']['data']))

In [ ]:
import sqlite3

In [ ]:
insert = "INSERT OR IGNORE INTO FILES (ibge, procurement_id, file_id, path, OCR_RAW, name, date) VALUES (?, ?, ?, ?, '', ?, ?)"
con = sqlite3.connect("scraped.db", timeout=50)
cursor = con.cursor()
cursor.executemany(insert, to_insert)
con.commit()
con.close()

In [ ]:
pip install aiofile

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa/PR')

Scraping.crawlers.crawler_oxy


In [ ]:
rows = cw.query_files(5)
rows

[(4119905, '1/2015/6', 6429),
 (4119905, '1/2015/7', 6715),
 (4119905, '1/2015/9', 6729),
 (4119905, '1/2016/6', 6099),
 (4119905, '1/2016/7', 6139)]

In [ ]:
import os
rows = cw.query_files()

i = 1
to_update = []
for row in rows:   
    file = os.path.join("", "Data", row[0].__str__(), row[1].replace("/", "-"))
    to_update.append((path, row[0], row[1], row[2]))

In [ ]:
insert = "UPDATE FILES set path = ? where ibge = ? and procurement_id = ? and file_id = ?"
con = sqlite3.connect("scraped.db", timeout=50)
cursor = con.cursor()
cursor.executemany(insert, to_update)
con.commit()
con.close()

In [ ]:
#Validar pastas
pastas = cw.query_folders()
for pasta in pastas:
    if not os.path.exists(pasta[0]):
        os.makedirs(pasta[0]) 

In [ ]:
files = cw.query_files(3)
files2 = [('https://{base}/portaltransparencia-api/api/files/arquivo/{file_id}'.format(
                base=cw._url,
                file_id=x[2]
            ), ) + x for x in files]
files2

[('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6429',
  4119905,
  '1/2015/6',
  6429,
  'Data\\4119905\\1-2015-6',
  '6429.pdf'),
 ('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6715',
  4119905,
  '1/2015/7',
  6715,
  'Data\\4119905\\1-2015-7',
  '6715.pdf'),
 ('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6729',
  4119905,
  '1/2015/9',
  6729,
  'Data\\4119905\\1-2015-9',
  '6729.pdf')]

In [ ]:
for file in files2:
    print(file)

('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6729', 4119905, '1/2015/9', 6729, 'Data\\4119905\\1-2015-9', '6729.pdf')


In [ ]:
pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#Check if file exists
files3 = [x for x in files2 if not os.path.isfile(os.path.join(x[4], x[5]))]

In [ ]:
#baixar e salvar cada arquivo

import os
import asyncio
import aiohttp  # pip install aiohttp
import aiofile  # pip install aiofile
import nest_asyncio

nest_asyncio.apply()

def download_files_from_report(files):

    #sema = asyncio.BoundedSemaphore(5)

    async def fetch_file(session, file):
        
        url = file[0]
        folder = file[4]
        fname = file[5]

        #async with sema:
        async with session.get(url) as resp:
            assert resp.status == 200
            data = await resp.read()

        async with aiofile.async_open(
            os.path.join(folder, fname), "wb"
        ) as outfile:
            await outfile.write(data)

    async def main():
        async with aiohttp.ClientSession() as session:
            tasks = [fetch_file(session, file) for file in files]
            await asyncio.gather(*tasks)

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
    #loop.close()

download_files_from_report(files3)

In [ ]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa/PR')
try:
    cw.get_all_files()
except:
    pass

Scraping.crawlers.crawler_oxy
there is 2483 files in the database
there is 1827 files to be downloaded
the first file id is: 6139.
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded


IndexError: list index out of range

In [2]:
pip install IronPdf --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
os.stat(r'Files\4119905\25-2020-6\22439.pdf').st_size

0

In [7]:
cw.do_ocr(1)

In [8]:
cw.query_files_to_do(1)

[]

In [9]:
cw.manual_update_file('23/2022/6', 1001379, '<Ignore>')

In [10]:
cw.check_empty()

AttributeError: 'CrawlerOxy' object has no attribute 'check_empty'

In [ ]:
import time
import math

i = 20
total = cw.query_qtd_files_to_do()[0][0]
loops = math.ceil(total / i )
print(f"Blocks: {loops}, Size: {i}, Total: {total}")

for l in range(loops):
    start_time = time.time()
    cw.do_ocr(i)
    t = str(((time.time() - start_time)/60))
    print(f"--- The block {str(l + 1)}, Doing {str(i)} files, took: {t} minutes ---")


Blocks: 0, Size: 20, Total: 0


In [ ]:
pip install python-docx 

     ------------------------------------ 239.6/239.6 kB 735.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import docx

def getText(filename):
    doc = docx.Document(filename)
    return '\n'.join([p.text for p in doc.paragraphs])
getText(r"Files\4119905\1-2023-6\1002096.docx")

'PA – Protocolo Geral: 51/2023\t\t\tEmissão: 10/01/2023\n\nData da abertura das propostas: 07/03/2023\tHorário: 09hs 30min\n\nPregoeiro: CHARLES METZGER FERREIRA\n\nA CÂMARA MUNICIPAL DE PONTA GROSSA, mediante o Pregoeiro designado pelo Ato 32/2023, torna público para conhecimento dos interessados, que na data e local abaixo indicados fará realizar licitação na modalidade PREGÃO ELETRÔNICO do tipo MENOR PREÇO GLOBAL, a contratação de empresa habilitada para fornecimento de “linha de Internet com modem grátis, que suporte download mínimo de 400 mega, upload mínimo de 200 mega com wi-fi inclusos e tecnologia e fibra óptica”, nos moldes da Lei nº 10.520/02, Decreto nº 10.024/19, Lei Complementar nº 123/06 e subsidiariamente, a Lei Federal nº 8.666/93, todas com as alterações posteriores e legislação correlata, a fim de escolher a melhor proposta.\n\nRECEBIMENTO DAS PROPOSTAS: Das 13:00 horas do dia 23/02/2023 às 09:00 horas do dia 08/03/2023.\n\nABERTURA E JULGAMENTO DAS PROPOSTAS: Das 09

In [3]:
cw.manual_update_file("2/2020/7", 22483, "<Ignore>")

In [1]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa', 'PR')

import nest_asyncio
nest_asyncio.apply()

## __init__ crawlers load: oxy
## Starting Scraping.crawlers.crawler_oxy ##
## Starting Scraping.crawlers.crawler_base ##
## __init__ crawlers load: base
## __init__ crawlers load: master
## Starting Scraping.crawlers.crawler_master ##


In [1]:
import torch
from transformers import AutoTokenizer,AutoModelForPreTraining, AutoModel

model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
input_ids = tokenizer.encode('Tinha uma pedra no meio do caminho.', return_tensors='pt')

with torch.no_grad():
    outs = model(input_ids)
    encoded = outs[0][0, 1:-1]

last_hidden_states = outs[0]

# Print the output features
print(last_hidden_states)

c:\Users\danie\OneDrive\7 - Estudos\3 - MBA USP 2022\6 - TCC\Code\GovBrProcurement\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[[ 0.2824, -0.1887,  0.5604,  ..., -0.6297,  0.0302, -0.5707],
         [-0.0398, -0.3057,  0.2431,  ..., -0.5420,  0.1857, -0.5775],
         [-0.2926, -0.1957,  0.7020,  ..., -0.2843,  0.0530, -0.4304],
         ...,
         [ 0.0662,  0.2845,  0.1871,  ..., -0.2542, -0.2933, -0.0661],
         [ 0.2761, -0.1657,  0.3288,  ..., -0.2102,  0.0029, -0.2009],
         [ 0.2825, -0.1884,  0.5614,  ..., -0.6288,  0.0294, -0.5712]]])


In [4]:
encoded.size()

torch.Size([8, 768])

In [2]:
cw.create_city()

IntegrityError: UNIQUE constraint failed: CITYS.ibge